# TODO
* value can contain comma-separated values (i.e. 375,87)
```
UPDATE your_table
SET values = REPLACE(values, ',', '.');
```

In [2]:
import requests
import pandas as pd
import folium
import sqlite3
import io

# to show all rows and columns 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format

In [3]:
def dict_factory(cursor, row):
    fields = [column[0] for column in cursor.description]
    return {key: value for key, value in zip(fields, row)}

In [4]:
import math

def distance(origin, destination):
    """
    Calculate the Haversine distance.

    Parameters
    ----------
    origin : tuple of float
        (lat, long)
    destination : tuple of float
        (lat, long)

    Returns
    -------
    distance_in_km : float

    Examples
    --------
    >>> origin = (48.1372, 11.5756)  # Munich
    >>> destination = (52.5186, 13.4083)  # Berlin
    >>> round(distance(origin, destination), 1)
    504.2
    """
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d

distance((52.2296756, 21.0122287), (52.406374, 16.9251681))

278.45817507541943

# DB Connect

In [5]:
con = sqlite3.connect("pegelstaende.db")
con.row_factory = dict_factory
cur = con.cursor()
con.execute('PRAGMA foreign_keys = ON;')

In [38]:
def run(sql):
    r = cur.execute(sql)
    return(r.fetchall())

In [39]:
run("""UPDATE timeseries
SET value = REPLACE(value, ',', '.');""")

[]

# API Call

In [15]:
# url = "https://pegelonline.wsv.de/webservices/rest-api/v2/stations.json"
# params = {"includeTimeseries": "true", "includeCharacteristicValues": "true"}
url = "https://pegelonline.wsv.de/webservices/rest-api/v2/stations.json?includeTimeseries=true&includeCharacteristicValues=true"
response = requests.get(url)

if response.status_code == 200:
    df = pd.DataFrame(response.json())
else:
    print("Error: Failed to fetch data from API")

# Filter out rows with missing latitude values
#df = df.dropna(subset=['longitude', 'latitude'])

df['shortname'] = df["shortname"].apply(lambda x: x.title())
df['longname'] = df["longname"].apply(lambda x: x.title())
df['fluss'] = df['water'].apply(lambda x: x['longname'].title())

In [20]:

#display(df[df.latitude.isna()])
df[df.latitude.isna()].shape


(146, 11)

In [21]:
df[df.shortname == "Emden Neue Seeschleuse"]

,uuid,number,shortname,longname,km,agency,longitude,latitude,water,timeseries,fluss
173,edfdf747-be92-462f-87ed-53d228a33172,3970010,Emden Neue Seeschleuse,Emden Neue Seeschleuse,40.45,STANDORT EMDEN,7.19,53.34,"{'shortname': 'EMS', 'longname': 'EMS'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Ems


In [22]:
df[df['fluss'] == "Main-Donau-Kanal"]

,uuid,number,shortname,longname,km,agency,longitude,latitude,water,timeseries,fluss
281,ff02f181-491c-4925-ad13-07edd2ddb3f1,24300042,Bamberg,Bamberg,7.31,STANDORT NÜRNBERG,10.91,49.88,"{'shortname': 'MDK', 'longname': 'MAIN-DONAU-K...","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Main-Donau-Kanal
282,4a69e82e-97a3-4573-8aeb-b695c1eaa0b1,13409200,Riedenburg_Up,Riedenburg_Up,151.20,STANDORT NÜRNBERG,11.69,48.97,"{'shortname': 'MDK', 'longname': 'MAIN-DONAU-K...","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Main-Donau-Kanal


# Fixing Values

In [23]:
### Fix for Weser
### The km for Weser is reset to 0 in the middle of the set from BREMEN

filter_condition = ((df["agency"] == "BREMEN") | (df["agency"] == "BREMERHAVEN" )) & (df["km"] < 350) & (df["fluss"] == "Weser")
df.loc[filter_condition, "km"] += 370

### Fix for Weser
### The km for Weser is reset to 0 in the middle of the set from BREMEN
df.loc[df.longname == "Herbrum Oberwasser", "km"] = 245
df.loc[df.longname == "Herbrum Unterwasser", "km"] = 246
filter_condition = (df["agency"] == "STANDORT EMDEN") & (df["km"] < 100) & (df["fluss"] == "Ems")
df.loc[filter_condition, "km"] += 250

# TODO
display(df[df.km < 0])


,uuid,number,shortname,longname,km,agency,longitude,latitude,water,timeseries,fluss
67,8e326085-c879-4704-95b4-c1fc19874176,061000,Prelouc,Prelouc,-220.10,POVODÍ LABE,NaN,NaN,"{'shortname': 'ELBE', 'longname': 'ELBE'}","[{'shortname': 'Q', 'longname': 'ABFLUSS_ROHDA...",Elbe
238,8122b09b-a174-41d9-9a38-c0a5223366b8,1020,Predmerice,Predmerice,-137.70,POVODÍ LABE,NaN,NaN,"{'shortname': 'Jizera', 'longname': 'JIZERA'}","[{'shortname': 'Q', 'longname': 'ABFLUSS_ROHDA...",Jizera
244,4e7a6cfa-7548-4f7f-a97a-eb0694881003,25830056,Marburg,Marburg,-38.70,MARBURG,NaN,NaN,"{'shortname': 'LAHN', 'longname': 'LAHN'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Lahn
245,4b386a6a-996e-4a4a-a440-15d6b40226d4,25800100,Giessen Klärwerk,Giessen Klärwerk,-3.20,STANDORT KOBLENZ,8.65,50.58,"{'shortname': 'LAHN', 'longname': 'LAHN'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Lahn
440,18e6fa6e-2564-4057-8503-05213e49e57a,2190,Louny,Louny,-61.40,POVODÍ LABE,NaN,NaN,"{'shortname': 'EGER', 'longname': 'OHRE'}","[{'shortname': 'Q', 'longname': 'ABFLUSS_ROHDA...",Ohre
446,04cbc946-8ee9-4521-ac52-557642cad52c,0370,Tyniste,Tyniste,-263.80,POVODÍ LABE,NaN,NaN,"{'shortname': 'Orlice', 'longname': 'ORLICE'}","[{'shortname': 'Q', 'longname': 'ABFLUSS_ROHDA...",Orlice
496,b2e89aef-c870-4c73-9112-0115b9120458,2390,Benesov,Benesov,-10.70,POVODÍ LABE,NaN,NaN,"{'shortname': 'Ploucnice', 'longname': 'PLOUCN...","[{'shortname': 'Q', 'longname': 'ABFLUSS_ROHDA...",Ploucnice
637,12deef85-1779-4b4f-9ba6-8467dc9d9866,2030,Vranany,Vranany,-105.90,POVODÍ LABE,NaN,NaN,"{'shortname': 'MOLDAU', 'longname': 'VLATAVA'}","[{'shortname': 'Q', 'longname': 'ABFLUSS_ROHDA...",Vlatava


## Empty km

In [24]:
# TODO OSTSEE shouldn't have km information
df[df.km.isna()]

,uuid,number,shortname,longname,km,agency,longitude,latitude,water,timeseries,fluss
66,6da1a0e2-9af5-4a4c-b52d-c0faead67d70,71161198,Hahnöfer Nebenelbe 1,Hahnöfer Nebenelbe 1,NaN,STANDORT HAMBURG,NaN,NaN,"{'shortname': 'ELBE', 'longname': 'ELBE'}","[{'shortname': 'LF', 'longname': 'ELEKTRISCHE_...",Elbe
167,86f8dbab-6a64-408b-a5d5-69e69f01db2f,3730001,Versen Wehr Op,Versen Wehr Op,NaN,STANDORT MEPPEN,NaN,NaN,"{'shortname': 'EMS', 'longname': 'EMS'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Ems
456,b8d05bf9-2ef2-416d-ba70-b5dba74fe777,9650024,Althagen,Althagen,NaN,STANDORT STRALSUND,12.42,54.37,"{'shortname': 'OSTSEE', 'longname': 'OSTSEE'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Ostsee
457,09227288-8c72-4b12-88fe-8765d1b2b400,9650040,Barhöft,Barhöft,NaN,STANDORT STRALSUND,13.03,54.43,"{'shortname': 'OSTSEE', 'longname': 'OSTSEE'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Ostsee


# Schleusen

In [26]:
import requests

url = "https://www.elwis.de/DE/dynamisch/mvc/main.php?modul=schleuseninfo&choice=1&show_contacts=1&show_sperr=1"

response = requests.get(url)

html_content = response.text

#print(html_content)

# <td class="b2"><a href="/DE/dynamisch/mvc/main.php?modul=schleuseninfo&amp;choice=1#s_144">Bannetze</a></td><td class="b2">0,5</td><td class="b2">An der Schleuse 1</td><td class="b2">29308</td><td class="b2">Bannetze</td><td class="b2" nowrap="nowrap">52° 40.874' N<br />9° 49.574' E</td><td class="b2">nicht angegeben</td><td class="b2" nowrap="nowrap">0163 6898849</td><td class="b2" nowrap="nowrap">nicht angegeben</td><td class="b2" nowrap="nowrap">nicht angegeben</td><td class="b2">selbstbedient</td><td class="b2"><a href="https://www.wsa-weser.wsv.de" target="_blank">WSA Weser</a></td></tr></table>


In [27]:
import re
import html

# Read the HTML file as text
with open('schleusen.html', 'r') as file:
    html_content = file.read()

# Define the pattern using regular expressions
pattern = r'<tr><td class="b2">(<a.*?</tr>)|<h1><a name="w_(\d+)">Schleusen an der Wasserstra&szlig;e (.*?)\n?</a>'

# Find all matches in the HTML content
matches = re.findall(pattern, html_content, re.DOTALL)

# <th class="b1" nowrap>Schleuse</th>
# <th class="b1" nowrap>km</th>
# <th class="b1" nowrap>Stra&szlig;e</th>
# <th class="b1" nowrap>PLZ</th>
# <th class="b1" nowrap>Ort</th>
# <th class="b1" nowrap>Koordinaten</th>
# <th class="b1" nowrap>Funkkanal</th>
# <th class="b1" nowrap>Telefon</th>
# <th class="b1" nowrap>Telefax</th>
# <th class="b1" nowrap>Telefon 2</th>
# <th class="b1" nowrap>Betriebsart</th>
# <th class="b1" nowrap><acronym title="Wasser- und Schifffahrtsamt">WSA</acronym></th>

columns = ['URL', "ID", 'Name', 'km', 'Adresse', 'PLZ', 'Ort', 'Koordinaten', 'Latitude', 'Longitude', 'Funkkanal', 'Telefon', 'Telefax', 'Telefon 2', 'Betriebsart', 'Link', 'WSA']

i = 0
matched_items = []
# Iterate over the found entries
for match in matches:
    #print(match)
    if match[0] == "":
        w_id = match[1]
        w_name = match[2]
    else:
        text = html.unescape(match[0])
        pattern = r'<a href="([^"]*?(?:_(\d+))?)">(.*?)</a></td><td class="b2">(.*?)</td><td class="b2">(.*?)</td><td class="b2">(.*?)</td><td class="b2">(.*?)</td><td class="b2" nowrap="nowrap">((.*?)?<br />(.*?)|.*?)</td><td class="b2">(.*?)</td><td class="b2" nowrap="nowrap">(.*?)</td><td class="b2" nowrap="nowrap">(.*?)</td><td class="b2"(?: nowrap="nowrap")?>(.*?)</td><td class="b2">(.*?)</td><td class="b2"><a href="(.*?)" target="_blank">(.*)</a></td></tr>'
        content = re.findall(pattern, text)
        data_dict = dict(zip(columns, content[0]))

        if data_dict["Name"] == "Fankel kleine Kammer":
            data_dict["Latitude"] = "50° 7.460' N"
            data_dict["Longitude"] = "7° 13.715' E"

        if data_dict["Latitude"] != "":
            degrees, minutes = data_dict["Latitude"].split('°')
            minutes, _ = minutes.split("'")
            degrees = float(degrees.strip())
            minutes = float(minutes.strip())
            data_dict["Latitude"] = degrees + (minutes / 60)

            degrees, minutes = data_dict["Longitude"].split('°')
            minutes, _ = minutes.split("'")
            degrees = float(degrees.strip())
            minutes = float(minutes.strip())
            data_dict["Longitude"] = degrees + (minutes / 60)
        elif data_dict["Name"] == "Augst":
            data_dict["Latitude"] = "47.5364551"
            data_dict["Longitude"] = "7.7090737"

        elif data_dict["Name"] == "Oslebshausen":
            data_dict["Latitude"] = "53.1198481"
            data_dict["Longitude"] = "8.7086116"
            
        else:
            print("No geo locations")
            display(data_dict)

        data_dict["Fluss_ID"] = w_id
        data_dict["Fluss"] = w_name

        matched_items.append(data_dict)

    i += 1
    #break

schleusen = pd.DataFrame(matched_items)



display(schleusen)

,URL,ID,Name,km,Adresse,PLZ,Ort,Koordinaten,Latitude,Longitude,Funkkanal,Telefon,Telefax,Telefon 2,Betriebsart,Link,WSA,Fluss_ID,Fluss
0,/DE/dynamisch/mvc/main.php?modul=schleuseninfo...,144,Bannetze,"0,5",An der Schleuse 1,29308,Bannetze,52° 40.874' N<br />9° 49.574' E,52.68,9.83,nicht angegeben,0163 6898849,nicht angegeben,nicht angegeben,selbstbedient,https://www.wsa-weser.wsv.de,WSA Weser,222,Aller Schleusenkanal Bannetze
1,/DE/dynamisch/mvc/main.php?modul=schleuseninfo...,146,Hademstorf,"0,5",Hansadamm 44,29693,Hademstorf,52° 42.749' N<br />9° 36.469' E,52.71,9.61,nicht angegeben,0163 6898849,nicht angegeben,nicht angegeben,vor Ort bedient,https://www.wsa-weser.wsv.de,WSA Weser,224,Aller Schleusenkanal Hademstorf
2,/DE/dynamisch/mvc/main.php?modul=schleuseninfo...,145,Marklendorf,"0,6",Stillenhöfen,29690,Essel/OT Stillenhöfen,52° 40.956' N<br />9° 42.240' E,52.68,9.70,nicht angegeben,0163 6898849,nicht angegeben,nicht angegeben,vor Ort bedient,https://www.wsa-weser.wsv.de,WSA Weser,223,Aller Schleusenkanal Marklendorf
3,/DE/dynamisch/mvc/main.php?modul=schleuseninfo...,143,Oldau,"0,4",Schleusenstraße 9,29313,Oldau,52° 39.609' N<br />9° 56.058' E,52.66,9.93,nicht angegeben,0163 6898849,nicht angegeben,nicht angegeben,selbstbedient,https://www.wsa-weser.wsv.de,WSA Weser,221,Aller Schleusenkanal Oldau
4,/DE/dynamisch/mvc/main.php?modul=schleuseninfo...,148,Plötzensee,"7,5",Nordufer 37,13351,Berlin,52° 32.586' N<br />13° 19.435' E,52.54,13.32,22,0303435710,03034357122,nicht angegeben,fernbedient,https://www.wsa-spree-havel.wsv.de,WSA Spree-Havel,92,Berlin-Spandauer-Schifffahrtskanal
5,/DE/dynamisch/mvc/main.php?modul=schleuseninfo...,149,Plötzensee Nordkammer,"7,5",Nordufer 37,13351,Berlin,52° 32.586' N<br />13° 19.435' E,52.54,13.32,22,0303435710,03034357122,nicht angegeben,fernbedient,https://www.wsa-spree-havel.wsv.de,WSA Spree-Havel,92,Berlin-Spandauer-Schifffahrtskanal
6,/DE/dynamisch/mvc/main.php?modul=schleuseninfo...,150,Plötzensee Südkammer,"7,5",Nordufer 37,13351,Berlin,52° 32.586' N<br />13° 19.435' E,52.54,13.32,22,nicht angegeben,nicht angegeben,nicht angegeben,fernbedient,Not_Found,Generaldirektion Wasserstraßen und Schifffahrt,92,Berlin-Spandauer-Schifffahrtskanal
7,/DE/dynamisch/mvc/main.php?modul=schleuseninfo...,541,Sportbootschleuse Brandenburg,"57,9",An der Stadtschleuse 5C,14776,Brandenburg an der Havel,52° 24.364' N<br />12° 33.418' E,52.41,12.56,nicht angegeben,+49 3381 226963,+49 3381 226963,nicht angegeben,vor Ort bedient,https://www.wsa-spree-havel.wsv.de,WSA Spree-Havel,85,Brandenburger Stadtkanal
8,/DE/dynamisch/mvc/main.php?modul=schleuseninfo...,151,Neue Mühle,"9,5",Tiergartenstr. 11a,15711,Königs Wusterhausen,52° 17.824' N<br />13° 39.056' E,52.30,13.65,nicht angegeben,+49 3375 293686,+49 3375 293686,nicht angegeben,vor Ort bedient,https://www.wsa-spree-havel.wsv.de,WSA Spree-Havel,113,Dahme-Wasserstraße
9,/DE/dynamisch/mvc/main.php?modul=schleuseninfo...,154,Hamm,"37,0",An der Schleuse,59065,Hamm,51° 41.186' N<br />7° 49.264' E,51.69,7.82,18,+49 2381 9019-280,+49 2381 9019-280,nicht angegeben,vor Ort bedient,https://www.wsa-westdeutsche-kanaele.wsv.de,WSA Westdeutsche Kanäle,11,Datteln-Hamm-Kanal


In [28]:
display(schleusen[schleusen.ID == "657"])

,URL,ID,Name,km,Adresse,PLZ,Ort,Koordinaten,Latitude,Longitude,Funkkanal,Telefon,Telefax,Telefon 2,Betriebsart,Link,WSA,Fluss_ID,Fluss
597,/DE/dynamisch/mvc/main.php?modul=schleuseninfo...,657,Oslebshausen,"8,6",nicht angegeben,nicht angegeben,nicht angegeben,ungültige Eingabe,53.1198481,8.7086116,nicht angegeben,nicht angegeben,nicht angegeben,nicht angegeben,nicht angegeben,Not_Found,Hansestadt Bremisches Hafenamt (HBH),171,Weser (Stadtbremische Häfen)


In [29]:
display(df.head(1))

,uuid,number,shortname,longname,km,agency,longitude,latitude,water,timeseries,fluss
0,47174d8f-1b8e-4599-8a59-b580dd55bc87,48900237,Eitze,Eitze,9.56,VERDEN,9.28,52.90,"{'shortname': 'ALLER', 'longname': 'ALLER'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Aller


# Plot Folium Map

In [30]:
map = folium.Map(location=[51.1657, 10.4515], zoom_start=6)

for group_name, group_data in df[~df.latitude.isna()].groupby("fluss"):
    # Create a feature group for the group
    group = folium.FeatureGroup(name=group_name)
    if group_name != "XEMS":
        locations = []

        for index, row in group_data.sort_values("km").iterrows():
            locations.append((row['latitude'], row['longitude']))
            folium.Marker(row[['latitude', 'longitude']], 
                          popup=group_name.title() + "  (" + row['longname'].title() + ") [" + row['agency'] + "]" + str(row['number']) + " / " + str(row['km']),
                          icon=folium.Icon(color='blue', icon='info-sign')).add_to(group)
        
        if group_name not in ( "Nordsee", "Ostsee" ):
            folium.PolyLine(locations, color='blue', weight=2.5).add_to(group)

        # Add the group to the map
        group.add_to(map)

group = folium.FeatureGroup(name="_Schleusen")
for schleuse, data in schleusen.groupby("ID"):
    name = data.Name.values[0]
    folium.Marker(data[['Latitude', 'Longitude']], 
                  popup=name + " (" + data.Fluss.values[0] + ")", 
                  icon=folium.Icon(color='red', icon='glyphicon-sort')).add_to(group)

# Add the group to the map
group.add_to(map)

# Add layer control to the map
folium.LayerControl().add_to(map)

# Display the map
map


In [36]:
df[df.fluss == "Elbe"]

,uuid,number,shortname,longname,km,agency,longitude,latitude,water,timeseries,fluss
66,6da1a0e2-9af5-4a4c-b52d-c0faead67d70,71161198,Hahnöfer Nebenelbe 1,Hahnöfer Nebenelbe 1,NaN,STANDORT HAMBURG,NaN,NaN,"{'shortname': 'ELBE', 'longname': 'ELBE'}","[{'shortname': 'LF', 'longname': 'ELEKTRISCHE_...",Elbe
67,8e326085-c879-4704-95b4-c1fc19874176,061000,Prelouc,Prelouc,-220.10,POVODÍ LABE,NaN,NaN,"{'shortname': 'ELBE', 'longname': 'ELBE'}","[{'shortname': 'Q', 'longname': 'ABFLUSS_ROHDA...",Elbe
68,d634b809-286e-4c68-aa5b-49932bf34a97,501421,Wittenberggüte,Wittenberggüte,1.00,BFG KOBLENZ,NaN,NaN,"{'shortname': 'ELBE', 'longname': 'ELBE'}","[{'shortname': 'TR', 'longname': 'TRÜBUNG', 'u...",Elbe
69,7cb7461b-3530-4c01-8978-7f676b8f71ed,501010,Schöna,Schöna,2.05,STANDORT DRESDEN,14.24,50.88,"{'shortname': 'ELBE', 'longname': 'ELBE'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Elbe
70,85d686f1-55b2-4d36-8dba-3207b50901a7,501040,Pirna,Pirna,34.67,STANDORT DRESDEN,13.93,50.96,"{'shortname': 'ELBE', 'longname': 'ELBE'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Elbe
71,70272185-b2b3-4178-96b8-43bea330dcae,501060,Dresden,Dresden,55.63,STANDORT DRESDEN,13.74,51.05,"{'shortname': 'ELBE', 'longname': 'ELBE'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Elbe
72,24440872-5bd2-4fb3-8554-907b49816c49,501080,Meissen,Meissen,82.20,STANDORT DRESDEN,13.48,51.16,"{'shortname': 'ELBE', 'longname': 'ELBE'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Elbe
73,b04b739d-7ffa-41ee-9eb9-95cb1b4ef508,501110,Riesa,Riesa,108.40,STANDORT DRESDEN,13.29,51.31,"{'shortname': 'ELBE', 'longname': 'ELBE'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Elbe
74,16b9b4e7-be14-41fd-941e-6755c97276cc,501160,Mühlberg,Mühlberg,128.02,STANDORT DRESDEN,13.19,51.44,"{'shortname': 'ELBE', 'longname': 'ELBE'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Elbe
75,83bbaedb-5d81-4bc6-9f66-3bd700c99c1f,501261,Torgau,Torgau,154.15,STANDORT DRESDEN,13.01,51.55,"{'shortname': 'ELBE', 'longname': 'ELBE'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Elbe


## Check why there are 29 stations for Neckar in the df but only 9 in the db

In [33]:
df[df.fluss == "Neckar"].head()

,uuid,number,shortname,longname,km,agency,longitude,latitude,water,timeseries,fluss
359,25582d3f-dc5f-4c70-bd08-e84fd13201ca,23800900,Mannheim Neckar,Mannheim Neckar,3.10,STANDORT MANNHEIM,8.47,49.49,"{'shortname': 'NECKAR', 'longname': 'NECKAR'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Neckar
360,827b2685-47ec-44df-a90f-980f5e0c1591,23800760,Heidelberg Up,Heidelberg Up,26.10,STANDORT HEIDELBERG,8.72,49.41,"{'shortname': 'NECKAR', 'longname': 'NECKAR'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Neckar
361,3944c1fd-39f5-47b1-bb6e-323db05d70e8,23800745,Ziegelhausen Ams,Ziegelhausen Ams,29.40,STANDORT HEIDELBERG,NaN,NaN,"{'shortname': 'NECKAR', 'longname': 'NECKAR'}","[{'shortname': 'Q', 'longname': 'ABFLUSS_ROHDA...",Neckar
362,4c00a166-7d6d-48d7-b4dc-673b96b4041e,23800690,Rockenau Ska,Rockenau Ska,60.70,STANDORT HEIDELBERG,9.01,49.44,"{'shortname': 'NECKAR', 'longname': 'NECKAR'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Neckar
363,fc1072e4-61d4-4e89-bc70-c7e8860fd208,23800620,Gundelsheim Up,Gundelsheim Up,93.80,STANDORT HEIDELBERG,9.15,49.28,"{'shortname': 'NECKAR', 'longname': 'NECKAR'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R...",Neckar


In [31]:
rivers = {}

for river, data in schleusen.groupby("Fluss"):
    rivers[river] = {}
    #rivers[river].append("Schleuse")
    #rivers[river]["Schleuse"] = data.shape[1]
    rivers[river]["Schleuse"] = data.shape[0]



for river, data in df.groupby("fluss"):
    if river not in rivers:
        rivers[river] = {}
    #rivers[river].append("Pegel")    
    rivers[river]["Pegel"] = data.shape[0]

In [32]:
display(rivers)

{'Aller Schleusenkanal Bannetze': {'Schleuse': 1},
 'Aller Schleusenkanal Hademstorf': {'Schleuse': 1},
 'Aller Schleusenkanal Marklendorf': {'Schleuse': 1},
 'Aller Schleusenkanal Oldau': {'Schleuse': 1},
 'Berlin-Spandauer-Schifffahrtskanal': {'Schleuse': 3, 'Pegel': 2},
 'Brandenburger Stadtkanal': {'Schleuse': 1},
 'Dahme-Wasserstraße': {'Schleuse': 1},
 'Datteln-Hamm-Kanal': {'Schleuse': 2, 'Pegel': 4},
 'Donau': {'Schleuse': 12, 'Pegel': 18},
 'Dortmund-Ems-Kanal': {'Schleuse': 35, 'Pegel': 10},
 'Elbe': {'Schleuse': 3, 'Pegel': 78},
 'Elbe-Havel-Kanal': {'Schleuse': 4, 'Pegel': 8},
 'Elbe-Lübeck-Kanal': {'Schleuse': 7, 'Pegel': 3},
 'Elbe-Seitenkanal': {'Schleuse': 6},
 'Ems': {'Schleuse': 10, 'Pegel': 17},
 'Finowkanal': {'Schleuse': 12, 'Pegel': 9},
 'Fulda': {'Schleuse': 5, 'Pegel': 4},
 'Havel-Oder-Wasserstraße': {'Schleuse': 5},
 'Havelkanal': {'Schleuse': 1, 'Pegel': 2},
 'Ilmenau': {'Schleuse': 3, 'Pegel': 8},
 'Küstenkanal': {'Schleuse': 2, 'Pegel': 1},
 'Lahn': {'Schleu

# Check content

In [79]:
from collections import defaultdict

total = 0
c = defaultdict(list)
#for uuid, station in df.groupby("uuid"):
# Iterate over every row in the DataFrame
for index, station in df.iterrows():

    if station["shortname"].lower() != station["longname"].lower():
        print("not equal")
        print(station["shortname"].lower(), "/", station["longname"].lower())

    timeseries_list = station['timeseries']
    
    # Iterate over every entry in the 'timeseries' list
    for ts in timeseries_list:
        if str(ts["longname"]) in c:
            c[str(ts["longname"])] += 1
        else:
            c[str(ts["longname"])] = 1
    #break
print(c)
print(index)

not equal
voßwinkel op / vosswinkel op
not equal
voßwinkel up / vosswinkel up
defaultdict(<class 'list'>, {'WASSERSTAND ROHDATEN': 647, 'WASSERTEMPERATUR ROHDATEN': 119, 'ELEKTRISCHE_LEITFÄHIGKEIT_ROHDATEN': 36, 'ABFLUSS': 19, 'DURCHFAHRTSHÖHE': 5, 'LUFTTEMPERATUR': 20, 'WASSERTEMPERATUR': 20, 'FLIESSGESCHWINDIGKEIT': 7, 'GRUNDWASSER ROHDATEN': 9, 'LUFTTEMPERATUR ROHDATEN': 41, 'WINDGESCHWINDIGKEIT': 22, 'LUFTFEUCHTE': 2, 'SAUERSTOFFGEHALT ROHDATEN': 9, 'TRÜBUNG': 6, 'ABFLUSS_ROHDATEN': 73, 'WINDRICHTUNG': 20, 'SALINITÄT': 1, 'TRÜBUNG_ROHDATEN': 2, 'FLIESSGESCHWINDIGKEIT_ROHDATEN': 2, 'RICHTUNGSTROM': 3, 'NIEDERSCHLAG': 1, 'NIEDRSCHLAGSINTENSITÄT': 1, 'WELLENPERIODE': 2, 'SIGNIFIKANTEWELLENHÖHE': 2, 'MAXIMALEWELLENHÖHE': 2, 'SAUERSTOFFGEHALT': 10, 'PH-WERT': 6, 'CHLORID': 6})
700


# DB

In [106]:
sql1 = """CREATE TABLE timeseries (
    uuid TEXT NOT NULL,
	date DATE NOT NULL,
    idx INT NOT NULL,
    type TEXT NOT NULL,
    key TEXT NOT NULL,
    value INT NOT NULL,
    
    PRIMARY KEY (date, type, idx, uuid)
)"""

sql2 = """CREATE TABLE stations (
    uuid TEXT PRIMARY KEY,
    river_id INT NOT NULL,
    number INT NOT NULL,
    name TEXT NOT NULL,
    km INT NOT NULL,
    agency TEXT NOT NULL,
    latitude INT NOT NULL,
    longitude INT NOT NULL,
    last_update DATE DEFAULT CURRENT_TIMESTAMP)"""

    #FOREIGN KEY (uuid) REFERENCES timeseries(uuid))

sql3 = """CREATE TABLE rivers (
	id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL)"""
# FOREIGN KEY (id) REFERENCES stations(river_id))

#cur.execute("DROP TABLE IF EXISTS timeseries")

cur.execute("DROP TABLE IF EXISTS stations")
cur.execute("DROP TABLE IF EXISTS rivers")
#cur.execute(sql1)
cur.execute(sql2)
cur.execute(sql3)

## Rivers

In [109]:
df["name"] = df['water'].apply(lambda x: x['longname'].title())
df["river_id"] = df.groupby(["name"]).ngroup() + 1
rivers = df[["name", "river_id"]].drop_duplicates()
rivers.rename(columns={"river_id": "id"}, inplace=True)
display(rivers.head())
rivers.to_sql("rivers", con, if_exists='append', index=False)

,name,id
0,Aller,1
5,Berlin-Spandauer-Schifffahrtskanal,2
7,Bodensee,3
8,Bützflether Süderelbe,4
9,Dahme-Wasserstrasse,5


99

## Stations

In [95]:
manual_matching = {
	'04790e03-4353-4e80-beef-b3ed439484bc': '53.7802743,9.0926564', # Neuenseer Schleusenfleth Siel
	'04cbc946-8ee9-4521-ac52-557642cad52c': '50.147456, 16.068702', # Tyniste
	'05ca0e33-5c71-4368-b824-2243fcff656c': '51.5480995,7.1579631', # Wanne Eickel Schleuse Ow
	'06c498e3-786a-48be-83be-6a6293ea3b8c': '51.6681772,7.355924', # Datteln Schleuse Uw
	'07374faf-2039-4430-ae6d-adc0e0784c4b': '50.1204703,8.912554', # Hanau Brücke Dfh
	'0d0082cf-0b4b-4c8b-8e97-5d37b511916b': '52.972278, 10.582736', # Uelzen Uw
	'1125a391-6586-4951-865f-3a2c72f2c81a': '52.390376, 8.191164', # Broxten
	'12a3037f-cbf3-49d3-8da5-77fb38730bba': '51.4442081,6.7171311', # Ruhrwehr Ow
	'12deef85-1779-4b4f-9ba6-8467dc9d9866': '50.316301, 14.362855', # Vranany
	'12f63015-c66a-4e89-a69b-ddd069872d40': '51.4813912,6.8170314', # Oberhausen Schleuse Uw
	'130a3761-e060-482e-944c-f2a75d6744b7': '48.7324881,9.2746979', # Esslingen Schleuse Up
	'14699a51-aaa5-4ae4-98a6-d35aa9a80ec3': '', # Hünxe Schleuse Uw
	'16578824-88de-4700-ab09-f61dbb1182bd': '', # Alken
	'18e6fa6e-2564-4057-8503-05213e49e57a': '', # Louny
	'1c80e441-a1d7-400c-b569-e038c9876a4f': '', # D1 Hanskalbsand Oberfläche
	'1cc71bb2-99e5-4479-8ae9-8375568ed22c': '', # Lisdorf Up
	'1e237d42-66cb-4145-ab99-c8fde205b281': '', # Westspange
	'20640279-2334-4acd-afb5-c8f3afc18379': '', # Sankt Aldegund Up
	'23045f8b-e7e8-4534-8162-b7dba81764db': '', # Wanne Eickel Schleuse Uw
	'259dcb4a-2366-45a6-a448-b06a8941ae16': '', # Lühort Lfk
	'27dfd137-3b88-42e8-9baf-6bee76e0717c': '', # Hann. List
	'2813f196-3404-4ac2-92be-e288fe1607af': '', # Ahsen Schleuse Ow
	'28ec91e8-90c0-44d1-8fd2-b0b64c00c43b': '', # Abbenfleth Sperrwerk
	'2f7cb553-73a0-4fb8-816e-af276ca4ccb7': '', # Lauffen Schleuse Up
	'31c32a38-aced-410c-b6a0-87506d14361f': '', # Esslingen Wehr Op
	'3944c1fd-39f5-47b1-bb6e-323db05d70e8': '', # Ziegelhausen Ams
	'3988cd01-1c7c-4bd3-a8ce-d324f73f346d': '', # D2 Juelssand Oberfläche
	'3dec3921-6f01-4801-b648-a7f3ee33acd3': '', # Parchim Güte
	'3eba7da6-fbd6-431c-91d3-522ec49275f0': '', # Uelzen
	'3ef81fc0-33dc-4f67-8bb8-3f66975292d5': '', # Kleinostheim_Wuk
	'427318d0-ee8d-4d9b-89f1-e8a8685b5d8c': '', # Gelsenkirchen Schleuse Uw
	'42b865fb-5dd8-4204-a7aa-260704d69e21': '', # Thune
	'42c11b7a-d5ff-4372-b5ee-3b755ea4cb6f': '', # Besigheim Wehr Up
	'456974a8-8cc8-4349-ba9f-b272ee6dcf3b': '', # Marbach Schleuse Up
	'48780245-36f2-453e-b430-5fa838851d6d': '', # Duisburg-Meiderich Schleuse Uw
	'4a2501bf-cb57-46bb-87d3-d6808286b31b': '', # Minsener Oog
	'4dbce62d-a015-4011-afcd-423bb8d4b7a1': '', # Hamm Ow
	'4dd743e9-3040-4f11-8c7c-d5c5789d80b4': '', # Datteln Schleuse Ow
	'4e7a6cfa-7548-4f7f-a97a-eb0694881003': '', # Marburg
	'4ed5f6da-b7a0-4431-ab1f-1a26e69a21bc': '', # Ruehen
	'4ef9dd9c-b32b-43fc-bc5e-3849d4ab7ecc': '', # Hamm Uw
	'50881379-70ca-4e85-9bf2-752a7e5aea8c': '', # Mettlach Up
	'516c4814-d70b-47bf-99bf-a76844b94e5e': '', # Lohnde
	'560cf185-0052-4e40-832b-7792b52dd343': '', # Kachlet Wehr Up
	'57db82c7-2785-44d0-bede-21de0686a6be': '', # Besigheim Schleuse Up
	'583420a8-8903-485e-8c35-003814506838': '', # Fankel Up
	'5918b484-597f-4a1c-8262-0d0cbbb7986d': '', # Templin Up
	'5b94e4b9-cc96-433f-abf7-08287137ec54': '', # Dornbusch Brücke
	'5b9f8134-8bf0-4b51-a548-72a36303bbc2': '', # Gelsenkirchen Schleuse Ow
	'5c1b6183-eaaf-436c-b863-8a191d9e1f5a': '', # Hünxe Schleuse Ow
	'5db8ab4b-315b-466e-b9d5-5ba5a19f5f08': '', # Dorsten Schleuse Uw
	'6288de60-38be-4291-9ff6-dcf6c0f58cea': '', # Rodenberger Aue-Ost
	'6610cf2e-7bfd-4686-b9ac-b105f492d708': '', # Dorsten Schleuse Ow
	'6760b547-a7e7-408a-b3aa-529fe376bfcd': '', # Bad Essen
	'69308142-f78e-4877-9af8-e7221b01d303': '', # Grevenmacher Up
	'69fb4730-4866-4838-9b18-2c0ee9dce80d': '', # Banzkow Up
	'6c5b6422-126d-4bf0-8856-7b86d74ea1b3': '', # Osloss
	'6c7c180f-6dc1-4c4d-97bd-814080df8bcf': '', # Hoerstel
	'6d3a3e2a-713c-4232-9b70-834a2c0e9705': '', # Minden
	'6d43098c-632d-4e89-996f-98b1c99e5630': '', # Twielenfleth Siel
	'6da1a0e2-9af5-4a4c-b52d-c0faead67d70': '', # Hahnöfer Nebenelbe 1
	'6e183f4b-a506-4706-aeed-cf33b34e76ad': '', # Pleidelsheim Schleuse Up
	'7068d262-f466-44e1-9bfa-832fcc16c433': '', # Cannstatt Schleuse Up
	'70bee932-1c14-42f8-b4df-9a85589ca80e': '', # Henrichenburg Ow
	'71357738-c8e2-4cb7-a41a-365659bb8d58': '', # Templin Op
	'728bd3e3-23f2-41c6-8ac5-4cfa223a5a7e': '', # Uelzen Ow
	'752a8c5a-5f2a-4575-af2f-d825cffc6eb3': '', # Ruthenstrom-Sperrwerk
	'7703d54d-ca33-44b9-b519-c618bae5bcc7': '', # Ahsen Schleuse Uw
	'7af19499-8bc8-4a91-964d-88618ea8998a': '', # Recke
	'7e530482-0a13-4728-99db-a376304398b8': '', # Strohauser Plate Ost
	'7fec2f4f-6a2e-47ec-8f3c-016c581e4bbd': '', # Artlenburg-Elk
	'80e002d8-bc6a-4685-9aab-88edbfc069e0': '', # Obertürkheim Schleuse Up
	'8122b09b-a174-41d9-9a38-c0a5223366b8': '', # Predmerice
	'81c95d3a-3193-4e9c-a411-adf97c751a43': '', # Hahnöfer Sand West Siel
	'824a046b-9ca3-4db8-993d-e755122854e0': '', # Ingolstadt Luitpoldstrasse
	'864a8111-d0c1-4b74-b94e-515f0869355e': '', # Nienbruegge
	'86f8dbab-6a64-408b-a5d5-69e69f01db2f': '', # Versen Wehr Op
	'892183d6-3a28-4347-afb4-bc89602ca927': '', # Suelfeld Uw
	'8a2e3048-7bb0-4d8e-bf8a-3fc85a2d4b89': '', # Beihingen Wehr Up
	'8bccd43b-ed6f-4ed8-ae6e-80ad7e330658': '', # Weser
	'8e326085-c879-4704-95b4-c1fc19874176': '', # Prelouc
	'90d92801-c13c-4b1a-8885-83d1a07d6db8': '', # Haldensleben
	'9427fafc-7146-4be1-a71e-4e4658236037': '', # Belum Pegel
	'98daae03-5aaa-4284-9717-7d52da4fe063': '', # Anderten
	'99dd338d-ef7d-43fd-a4e0-a9103406f0ad': '', # Blexen Abz
	'9cf795ce-8bdf-494b-b8c6-ef1bfb89c08b': '', # Wittingen
	'9de0843a-2b79-40e5-b312-31d86e91ce4d': '', # Oberesslingen Wehr Up
	'9dfb4b0b-daa6-4a90-9a82-60734d3d2f04': '', # Suelfeld
	'a392113c-d040-4b36-b47d-9099275bb308': '', # Wintrich Up
	'a5ca02fb-9a83-4da5-80f0-0e7007350af8': '', # Untertürkheim Schleuse Up
	'a7ffcc9e-fecd-431c-8044-e004682d023f': '', # Flaesheim Schleuse Uw
	'a8604e8f-9330-4431-8cf6-0a68fc793c82': '', # Suelfeld Ow
	'a98b4a84-e51f-4d03-86d9-3b8bdac10e69': '', # Friedrichsfeld Schleuse Uw
	'ab0d9117-83f3-412d-bce5-b3e698246c0b': '', # Sehnde
	'ac6c4362-1fb1-4b7a-9406-016bed804804': '', # Herne Ow
	'ac970eb0-f5cf-4db0-90f0-e281f5d4775c': '', # Hahlen
	'ade3a084-8680-47b5-9d2e-288cb6c5f9d5': '', # Bergeshövede
	'b2e89aef-c870-4c73-9112-0115b9120458': '', # Benesov
	'b76bf04c-45e7-40c7-8940-3e1419c6da15': '', # Horkheim Schleuse Up
	'b8997708-53d0-4932-8fbf-265083e11517': '', # Bevensen
	'b8b8c069-0993-4a45-a226-a6c8c18185ab': '', # Lemwerder Unterfeuer
	'b9975ea1-5b45-4e27-abb8-4c3f7878b1f7': '', # Herne Uw
	'bc20d819-1782-4588-885d-129f21a27cf9': '', # Anderten Ow
	'be05603a-83e4-48d4-90d9-e074b7d34dcd': '', # Mehrum
	'bec8a6a5-c3de-4322-856a-870ceb992fe4': '', # Bramsche
	'c0594fb5-77ff-4287-9b8d-7ff326afe9ff': '', # Hattingen
	'c1f06401-9488-4a27-b932-59cce52c5819': '', # Rusbend
	'c2aa9164-1073-4271-82ef-44344fbe456e': '', # Luebbecke
	'c5b6243d-5b7e-4ec6-be61-2009085388d0': '', # Deizisau Schleuse Up
	'c7364d1e-6139-4575-84cb-b420d21275c4': '', # Lueneburg Ow
	'cbfae6bc-7978-4aac-9207-7939c8080507': '', # Oberesslingen Schleuse Up
	'cc57198c-b40f-4c11-ad50-fceae4669a3e': '', # Schwinge-Sperrwerk
	'ccd3e8f1-39e9-4e09-aa41-625afda84460': '', # Münster Ow
	'cf2af1a4-41a9-4a48-871b-f241daad04cc': '', # Hofen Schleuse Up
	'd18033cd-9cf6-4800-9176-57758bb444dc': '', # Lueneburg Uw
	'd2c8f60e-fda4-4744-9017-9c727fdcfe1f': '', # Berenbusch
	'd31767ef-2cf8-4046-9881-d5651d81ade8': '', # Werries Ow
	'd3e21e8a-99ce-4033-8db6-9d0f0648beb3': '', # Freiburg Sperrwerk
	'd634b809-286e-4c68-aa5b-49932bf34a97': '', # Wittenberggüte
	'd9b441a5-a585-4ac4-ae65-351a5fce3aa5': '', # Horkheim Wehr Up
	'da453ad0-5f1d-417c-baa3-74ae297f0b7a': '', # Auheim Brücke Dfh
	'dbb2bb9f-0f80-4df4-9d5c-98af2ee41826': '', # Vorsfelde
	'dc1bb420-ec60-4bc3-bce2-b3869a2513df': '', # Lehmen Up
	'dc344605-c507-4054-b4e3-293cf0602bc4': '', # Detzem Up
	'dd41a7d6-9b11-4ec4-9518-388266750cfa': '', # Anderten Uw
	'dfdfb4ff-370c-4795-a44e-0c2503674932': '', # Aldingen Schleuse Up
	'e020e651-e422-46d3-ae28-34887c5a4a8e': '', # Konstanz-Rhein
	'e0747660-6d75-476a-be0f-73865bf47363': '', # Wischhafen Sperrwerk
	'e0d7d4b2-7a72-429f-a858-f1e6f5ff40ae': '', # Lt Alte Weser (Radar)
	'e251f943-ce2f-4003-b208-e2797aa72202': '', # Oberhausen Schleuse Ow
	'e3409c98-72d7-425b-9e2d-da54f75b3be8': '', # Friedrichsfeld Schleuse Ow
	'e6b68bc2-6814-4bbf-b37e-44a0b7807c2d': '', # Henrichenburg Uw
	'eb24b5a3-5102-429b-93a9-15656fee850f': '', # Rodenberger Aue-West
	'eb28e62a-5009-4e07-9d63-225e31abe62d': '', # Rehlingen Up
	'ed260406-bdd6-42ef-bf2a-1246eea392f9': '', # Münster Uw
	'edc67ca3-b597-402a-9037-ff39ee816452': '', # Müden Up
	'ee861944-647f-4242-bf61-60f9b9d75fb7': '', # Enkirch Up
	'eeaba884-d4c5-4a83-88fb-adcd79adbc50': '', # Hanweiler
	'f18e81da-1fec-4308-a881-6defa7c6d18b': '', # Duisburg-Meiderich Schleuse Ow
	'f348f2f6-37c2-41c2-8d53-fd06a18acce0': '', # Lueneburg
	'f44e7220-892e-40b5-8ef4-3da129c2a6ae': '', # Geversdorf Brücke
	'f4854a4c-490a-43df-bc06-d2711ddf9b70': '', # Poppenweiler Schleuse Up
	'f5bc4a51-004f-4e5e-99b9-e6ae888d57de': '', # Lüdinghausen
	'f833a494-961b-42d3-9843-e4e61607b5be': '', # Flaesheim Schleuse Ow
	'f973fb71-4a44-4603-8b13-25002dd6dbd8': '', # Schöneworth Siel
	'facc5c16-f700-4248-9893-a395bffaf2e4': '', # Waltrop
}

In [108]:
# CREATE TABLE stations (
#     uuid TEXT PRIMARY KEY,
#     river_id INT NOT NULL,
#     number INT NOT NULL,
#     name TEXT NOT NULL,
#     km INT NOT NULL,
#     agency TEXT NOT NULL,
#     latitude INT NOT NULL,
#     longitude INT NOT NULL,
#     last_update DATE DEFAULT CURRENT_TIMESTAMP
stations = df[["uuid", "river_id", "number", "km", "agency", "latitude", "longitude"]].drop_duplicates()
stations["name"] = df['longname'].apply(lambda x: x.title())
stations["agency"] = df['agency'].apply(lambda x: x.title())
#stations.rename(columns={"longname": "name"}, inplace=True)

display(stations[stations["uuid"].duplicated()])

#display(stations[stations.latitude.isna()]["uuid"])
# cur.execute("ALTER TABLE stations DROP FOREIGN KEY uuid")
stations[~(stations.latitude.isna() | stations.km.isna())].to_sql("stations", con, if_exists='append', index=False)

,uuid,river_id,number,km,agency,latitude,longitude,name


553

In [182]:
map = folium.Map(location=[51.1657, 10.4515], zoom_start=9)

for group_name, group_data in df.groupby(df['name']):
    # Create a feature group for the group
    group = folium.FeatureGroup(name=group_name)
    if group_name != "XAller":
        #display(group_data.sort_values("km"))#[["agency", "km", "longname"]])
        #break
        locations = []
        # Add markers for each point in the group


        sort_by = "number"
        # Print the sorted locations
        if group_name in [ "Elbe", "Ems", "Havel-Oder-Wasserstrasse", "Main", "Oder", "Spree-Oder-Wasserstrasse", "Untere Havel-Wasserstrasse"]:
            sort_by = "km"
        
        for index, row in group_data.sort_values(sort_by).iterrows():
            #print(loc["agency"], loc["km"], loc["longname"])
            locations.append((row['latitude'], row['longitude']))
            folium.Marker(row[['latitude', 'longitude']], popup=group_name + " / " + row['longname'].lower() + " / " + row['number'].lower() + " / " + str(row['km'])).add_to(group)
    
        #print(locations)
        # Create lines between points in the group
        #locations = group_data[['latitude', 'longitude', group_name[0]]].values.tolist()
        folium.PolyLine(locations, color='blue', weight=2.5).add_to(group)

        # Add the group to the map
        group.add_to(map)

# Add layer control to the map
folium.LayerControl().add_to(map)

# Display the map
map

## Dates

In [23]:
from datetime import datetime, timedelta

# Get the current date
current_date = datetime.now()

# Generate the previous 31 dates
previous_dates = [current_date - timedelta(days=i) for i in range(30)]

# Format the dates as "dd.mm.yyyy"
#formatted_dates = [date.strftime("%d.%m.%Y") for date in reversed(previous_dates)]
formatted_dates = [[date.strftime("%d.%m.%Y"), date.strftime("%Y-%m-%d")] for date in reversed(previous_dates)]

In [124]:
type = {}
for index, station in df.iterrows():
    timeseries_list = station['timeseries']

    for ts in timeseries_list:
        s = str(ts["shortname"])
        t = str(ts["longname"])
        if str(ts["longname"]) in type:
            type[str(ts["longname"])] += 1
        else:
            type[str(ts["longname"])] = 0
            if t in ("FLIESSGESCHWINDIGKEIT", "FLIESSGESCHWINDIGKEIT_ROHDATEN", "LUFTTEMPERATUR", "LUFTTEMPERATUR ROHDATEN", "LUFTFEUCHTE", "PH-WERT", 
                     "SIGNIFIKANTEWELLENHÖHE", "TRÜBUNG", "TRÜBUNG_ROHDATEN", "WASSERSTAND ROHDATEN", "WASSERTEMPERATUR", "WASSERTEMPERATUR ROHDATEN", "WASSERTEMPERATUR"):
                t = t.title()
                t = t.replace("Fliess", "Fließ")
                t = t.replace("Ph-", "PH-")
                t = t.replace("wellenhöhe", "Wellenhöhe")
            print(f"\t\"{s}\": \"{t}\",")

# display(type)

	"W": "Wasserstand Rohdaten",
	"WT": "Wassertemperatur Rohdaten",
	"LF": "ELEKTRISCHE_LEITFÄHIGKEIT_ROHDATEN",
	"Q": "ABFLUSS",
	"DFH": "DURCHFAHRTSHÖHE",
	"LT": "Lufttemperatur",
	"WT": "Wassertemperatur",
	"VA": "Fließgeschwindigkeit",
	"GRU": "GRUNDWASSER ROHDATEN",
	"LT": "Lufttemperatur Rohdaten",
	"WG": "WINDGESCHWINDIGKEIT",
	"HL": "Luftfeuchte",
	"O2": "SAUERSTOFFGEHALT ROHDATEN",
	"TR": "Trübung",
	"Q": "ABFLUSS_ROHDATEN",
	"WR": "WINDRICHTUNG",
	"S": "SALINITÄT",
	"TR": "Trübung_Rohdaten",
	"VA": "Fließgeschwindigkeit_Rohdaten",
	"R": "RICHTUNGSTROM",
	"NIEDERSCHLAG": "NIEDERSCHLAG",
	"NIEDERSCHLAGSINTENSITÄT": "NIEDRSCHLAGSINTENSITÄT",
	"TP": "WELLENPERIODE",
	"SIGH": "SignifikanteWellenhöhe",
	"MAXH": "MAXIMALEWELLENHÖHE",
	"O2": "SAUERSTOFFGEHALT",
	"PH": "PH-Wert",
	"CL": "CHLORID",


In [126]:
# https://pegelonline.wsv.de/webservices/files/Wasserstand%20Rohdaten/ELBE/70272185-b2b3-4178-96b8-43bea330dcae/31.05.2023/down.csv
url = "https://pegelonline.wsv.de/webservices/files/{type}/{fluss}/{station_id}/{datum}/down.csv"

typematch = {
    "W": "Wasserstand Rohdaten",
	"WT": "Wassertemperatur Rohdaten",
	"LF": "ELEKTRISCHE_LEITFÄHIGKEIT_ROHDATEN",
	"Q": "ABFLUSS",
	"DFH": "DURCHFAHRTSHÖHE",
	"LT": "Lufttemperatur",
	"WT": "Wassertemperatur",
	"VA": "Fließgeschwindigkeit",
	"GRU": "GRUNDWASSER ROHDATEN",
	"LT": "Lufttemperatur Rohdaten",
	"WG": "WINDGESCHWINDIGKEIT",
	"HL": "Luftfeuchte",
	"O2": "SAUERSTOFFGEHALT ROHDATEN",
	"TR": "Trübung",
	"Q": "ABFLUSS_ROHDATEN",
	"WR": "WINDRICHTUNG",
	"S": "SALINITÄT",
	"TR": "Trübung_Rohdaten",
	"VA": "Fließgeschwindigkeit_Rohdaten",
	"R": "RICHTUNGSTROM",
	"NIEDERSCHLAG": "NIEDERSCHLAG",
	"NIEDERSCHLAGSINTENSITÄT": "NIEDRSCHLAGSINTENSITÄT",
	"TP": "WELLENPERIODE",
	"SIGH": "SignifikanteWellenhöhe",
	"MAXH": "MAXIMALEWELLENHÖHE",
	"O2": "SAUERSTOFFGEHALT",
	"PH": "PH-Wert",
	"CL": "CHLORID"
}

skip = False
#for group_name, group_data in df.groupby([df['water'].apply(lambda x: x['longname'])]):
for index, station in df.iterrows():
    timeseries_list = station['timeseries']

    # if station["uuid"] == "47174d8f-1b8e-4599-8a59-b580dd55bc87":
    #     continue
    # elif station["uuid"] == "005dfdae-efcc-410a-bf1c-24096e0c2866":
    #     skip = False

    if skip == True:
        continue

    for ts in timeseries_list:
        if str(ts["shortname"]) == "W":
            continue
        
        t = typematch[str(ts["shortname"])]
        for date in formatted_dates:
            sql = """SELECT 
    *
FROM timeseries ts
WHERE uuid = '""" + station["uuid"] + """'
    AND type = '""" + str(ts["shortname"]) + """'
    AND date = '""" + date[-1] + """'
    AND value != 'XXX,XXX'"""

            check = (pd.DataFrame(run(sql)))

            if check.shape[0] == 96:
                continue
            elif check.shape[0] == 1440:
                continue
            # display(sql)
            # display(check.shape)
            # display(check.head(2))

            fluss=station["water"]["longname"].replace(' ', '+')

            if fluss in ["FINOWKANAL", "DORTMUND-EMS-KANAL", "KLEINES+HAFF", "LYCHENER+GEWÄSSER", "NIEGRIPPER+VERBINDUNGSKANAL", "NORD-OSTSEE-KANAL", "ORANIENBURGER+HAVEL", "ORANIENBURGER+KANAL", "RHEIN-HERNE-KANAL", "ROTHENSEER-VERBINDUNGSKANAL", "TEMPLINER+GEWÄSSER", "WENTOW-GEWÄSSER", "WERBELLINER+GEWÄSSER", "WESEL-DATTELN-KANAL"]:
                fluss = fluss.title()

            temp = url.format(station_id=station["uuid"], fluss=fluss, datum=date[0], type=t)
            # print(temp)

            response = requests.get(temp)

            if response.status_code == 200:
                content = io.StringIO(response.content.decode("utf-8"))
                data = pd.read_csv(content, delimiter=";", skiprows=1, on_bad_lines='skip', names=["key", "value"], header=None)
            else:
                print(f"Error: Failed to fetch data for type {t} from API", station.uuid, date[-1])
                continue

            data.reset_index(inplace=True)
            data.rename(columns={'index': 'idx'}, inplace=True)
            data["uuid"] = station["uuid"]
            data["type"] = str(ts["shortname"])
            data["date"] = date[-1]

            if check.shape[0] > 0 and check[check.value != 'XXX,XXX'].shape[0] > 0:
                checked = check["key"].tolist()
                data = data[~data['key'].isin(checked)]
                
                if data[data.value != 'XXX,XXX'].shape[0] > 0:
                    display(data[data.value != "XXX,XXX"].tail(2))

            if data[data.value != 'XXX,XXX'].shape[0] > 0:
                print(f"Import {data[data.value != 'XXX,XXX'].shape[0]} rows of type {t} for {station.uuid} on {date[-1]}")
                data[data.value != "XXX,XXX"].to_sql("timeseries", con, if_exists='append', index=False)
        

Import 96 rows of type Wasserstand Rohdaten for 47174d8f-1b8e-4599-8a59-b580dd55bc87 on 2023-05-26
Import 96 rows of type Wasserstand Rohdaten for 47174d8f-1b8e-4599-8a59-b580dd55bc87 on 2023-05-27
Import 96 rows of type Wasserstand Rohdaten for 47174d8f-1b8e-4599-8a59-b580dd55bc87 on 2023-05-28
Import 96 rows of type Wasserstand Rohdaten for 47174d8f-1b8e-4599-8a59-b580dd55bc87 on 2023-05-29
Import 96 rows of type Wasserstand Rohdaten for 47174d8f-1b8e-4599-8a59-b580dd55bc87 on 2023-05-30
Import 96 rows of type Wasserstand Rohdaten for 47174d8f-1b8e-4599-8a59-b580dd55bc87 on 2023-05-31
Import 96 rows of type Wasserstand Rohdaten for 47174d8f-1b8e-4599-8a59-b580dd55bc87 on 2023-06-01
Import 96 rows of type Wasserstand Rohdaten for 47174d8f-1b8e-4599-8a59-b580dd55bc87 on 2023-06-02
Import 96 rows of type Wasserstand Rohdaten for 47174d8f-1b8e-4599-8a59-b580dd55bc87 on 2023-06-03
Import 96 rows of type Wasserstand Rohdaten for 47174d8f-1b8e-4599-8a59-b580dd55bc87 on 2023-06-04
Import 96 

,idx,key,value,uuid,type,date
50,50,12:45,50,5aaed954-de4e-4528-8f65-f3f530bc8325,W,2023-06-13
51,51,13:00,50,5aaed954-de4e-4528-8f65-f3f530bc8325,W,2023-06-13


Import 24 rows of type Wasserstand Rohdaten for 5aaed954-de4e-4528-8f65-f3f530bc8325 on 2023-06-13


,idx,key,value,uuid,type,date
51,51,13:00,22,522286e2-b2b3-4d0d-9a11-01b3ea418c76,W,2023-06-13
52,52,13:15,22,522286e2-b2b3-4d0d-9a11-01b3ea418c76,W,2023-06-13


Import 23 rows of type Wasserstand Rohdaten for 522286e2-b2b3-4d0d-9a11-01b3ea418c76 on 2023-06-13


...

,idx,key,value,uuid,type,date
53,53,13:30,751,99619dc5-9c88-45e9-a97d-653c41cda8a1,W,2023-06-13
54,54,13:45,751,99619dc5-9c88-45e9-a97d-653c41cda8a1,W,2023-06-13


Import 25 rows of type Wasserstand Rohdaten for 99619dc5-9c88-45e9-a97d-653c41cda8a1 on 2023-06-13


,idx,key,value,uuid,type,date
52,52,13:15,303,c389c9e2-a5d8-4104-a4cf-510ade44f143,W,2023-06-13
53,53,13:30,302,c389c9e2-a5d8-4104-a4cf-510ade44f143,W,2023-06-13


Import 25 rows of type Wasserstand Rohdaten for c389c9e2-a5d8-4104-a4cf-510ade44f143 on 2023-06-13


OperationalError: database is locked

https://pegelonline.wsv.de/webservices/files/Wasserstand%20Rohdaten/DORTMUND-EMS-KANAL/04572010-1db2-4338-8562-b3dca5e715c5/01.05.2023/down.csv
Error: Failed to fetch data from API

https://pegelonline.wsv.de/webservices/files/Wasserstand%20Rohdaten/FINOWKANAL/3cd46cc7-b94d-424f-a860-c389dd9992a4/01.05.2023/down.csv
https://pegelonline.wsv.de/webservices/files/Wasserstand+Rohdaten/Finowkanal/3cd46cc7-b94d-424f-a860-c389dd9992a4/01.05.2023/down.csv

In [274]:
display(df.head(2))
print(data.head())

,uuid,number,shortname,longname,km,agency,longitude,latitude,water,timeseries
0,47174d8f-1b8e-4599-8a59-b580dd55bc87,48900237,EITZE,EITZE,9.56,VERDEN,9.28,52.90,"{'shortname': 'ALLER', 'longname': 'ALLER'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R..."
1,5aaed954-de4e-4528-8f65-f3f530bc8325,48900204,RETHEM,RETHEM,34.22,VERDEN,9.38,52.79,"{'shortname': 'ALLER', 'longname': 'ALLER'}","[{'shortname': 'W', 'longname': 'WASSERSTAND R..."


   idx    key  value                                  uuid type        date
0    0  00:15    296  47174d8f-1b8e-4599-8a59-b580dd55bc87    W  2023-05-01
1    1  00:30    296  47174d8f-1b8e-4599-8a59-b580dd55bc87    W  2023-05-01
2    2  00:45    296  47174d8f-1b8e-4599-8a59-b580dd55bc87    W  2023-05-01
3    3  01:00    296  47174d8f-1b8e-4599-8a59-b580dd55bc87    W  2023-05-01
4    4  01:15    296  47174d8f-1b8e-4599-8a59-b580dd55bc87    W  2023-05-01


In [2]:
# ALTER TABLE your_table DROP FOREIGN KEY your_fk_constraint_name;
#cur.execute("ALTER TABLE stations DROP FOREIGN KEY uuid")
#cur.execute("DELETE FROM timeseries WHERE uuid = 'ac507f42-1593-49ea-865f-10b2523617c7'")

In [164]:
url = "https://pegelonline.wsv.de/webservices/rest-api/v2/stations/{station_id}/{parameter}/measurements.json"
# https://pegelonline.wsv.de/webservices/files/Wasserstand%20Rohdaten/ELBE/70272185-b2b3-4178-96b8-43bea330dcae/31.05.2023/down.csv

check = False

for index, station in df.iterrows():
    timeseries_list = station['timeseries']
    if station["uuid"] != "47174d8f-1b8e-4599-8a59-b580dd55bc87":
        continue
    elif station["uuid"] != "ac507f42-1593-49ea-865f-10b2523617c7":
        check = True
    
    if check == False:
        continue
    
    for ts in timeseries_list:
        temp = url.format(station_id=station["uuid"], parameter=ts["shortname"])
        print(temp)
        break
        response = requests.get(temp)

        if response.status_code == 200:
            data = pd.DataFrame(response.json())
        else:
            print("Error: Failed to fetch data from API")
        break

https://pegelonline.wsv.de/webservices/rest-api/v2/stations/47174d8f-1b8e-4599-8a59-b580dd55bc87/W/measurements.json


In [1]:
# data.uuid = station["uuid"]
# display(data)

# DB Disconnect

In [ ]:
cur.close()
con.close()